In [1]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from datasets import load_dataset, Dataset
import numpy as np
import matplotlib.pyplot as plt

import json
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

c:\Users\sin2x\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка данных из JSON файла
def load_poems(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [3]:
# Подготовка данных для обучения
def prepare_dataset(poems):
    texts = []
    for poem in poems.values():
        texts.append("\n".join(poem))  # Объединяем строки стихотворения в один текст
    return Dataset.from_dict({"text": texts})

In [4]:
def plot_metrics(metrics):
    plt.plot(metrics['train_loss'], label='Train Loss')
    plt.plot(metrics['eval_loss'], label='Eval Loss')
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    
    plt.title('Training and Evaluation Loss')
    plt.legend()
    
    plt.show()

In [5]:

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Загрузка предобученной модели DeepPavlov GPT-2
model_name = "DeepPavlov/rudialogpt3_medium_based_on_gpt2_v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [6]:
from sklearn.model_selection import train_test_split
# Загрузка данных
poems = load_poems('data/poems.json')


# Токенизация
def tokenize_function(examples):
    tokens = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens
    
dataset = prepare_dataset(poems)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']



Map: 100%|██████████| 368/368 [00:00<00:00, 3067.43 examples/s]


In [ ]:
import random

delimiter_array = [
    "|0|", "|1|", "|2|", "|3|",
    "|0|0|", "|0|1|", "|0|2|", "|0|3|",
    "|1|0|", "|1|1|", "|1|2|", "|1|3|",
    "|2|0|", "|2|1|", "|2|2|", "|2|3|",
    "|3|0|", "|3|1|", "|3|2|", "|3|3|"
]

def generate_dispreferred_poem(model, input_ids, tokenizer):
    preferred_poem = tokenizer.decode(input_ids[0], skip_special_tokens=True)

    lines = preferred_poem.split('/n')
    lines = [line for line in lines if random.random() > 0.1]
    for line in lines:
        words = line.split()
        words = [word for word in words if random.random() > 0.05]
        line = ' '.join(words)
        
    result_lines = []

    lines.append("")
    skip = 1
    last_string = ""
    for line in lines:

        if skip==0 and random.random() < 0.4:
            combined_line = last_string.replace('\n', random.choice(delimiter_array)) +line
            result_lines.append(combined_line)
            skip = 1
        elif skip == 1:
            last_string = line
            skip = 0
        else: 
            result_lines.append(last_string)
            last_string = line
           
    final_poem = ' '.join(result_lines)

    if final_poem.strip() == "":
        return input_ids
    else:
        return tokenizer(final_poem, return_tensors='pt', padding=True).input_ids

In [ ]:
# Замораживаем параметры модели
for param in model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    learning_rate=2e-8,
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    logging_steps=20,
    save_strategy="epoch",
    save_total_limit=2,
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

import torch.nn.functional as F

def dpo_loss_fn(preferred_logits, dispreferred_logits):
    preferred_scores = preferred_logits.sum(dim=1)  
    dispreferred_scores = dispreferred_logits.sum(dim=1)  
    
    score_diff = preferred_scores - dispreferred_scores 
    
    loss = -F.logsigmoid(score_diff).mean()
    
    return loss

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
import torch
import torch.nn as nn

def training_step(self, model, inputs, num_items_in_batch=None):
    
    preferred_input_ids = inputs["input_ids"]
    
    dispreferred_input_ids = generate_dispreferred_poem(model, preferred_input_ids, tokenizer)
    
    preferred_outputs = model(**inputs)
    preferred_logits = preferred_outputs.logits
    
    dispreferred_outputs = model(input_ids=dispreferred_input_ids)
    dispreferred_logits = dispreferred_outputs.logits

    loss = dpo_loss_fn(preferred_logits, dispreferred_logits)
    return loss

trainer.training_step = training_step.__get__(trainer)

trainer.train()

c:\Users\sin2x\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,1015.743200,16.325445
2,972.634700,16.325445
3,991.081300,16.325445


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
c:\Users\sin2x\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sin2x\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=222, training_loss=968.683620143581, metrics={'train_runtime': 2426.9498, 'train_samples_per_second': 0.363, 'train_steps_per_second': 0.091, 'total_flos': 819114012573696.0, 'train_loss': 968.683620143581, 'epoch': 3.0})

In [10]:
# Сохранение модели после обучения
model.save_pretrained('./poem_generator')
tokenizer.save_pretrained('./poem_generator')

('./poem_generator\\tokenizer_config.json',
 './poem_generator\\special_tokens_map.json',
 './poem_generator\\vocab.json',
 './poem_generator\\merges.txt',
 './poem_generator\\added_tokens.json',
 './poem_generator\\tokenizer.json')

In [ ]:

reference_texts = list(poems.values()) 

generated_texts = []
model.eval()  

with torch.no_grad():
    for _ in range(10): 
        random_index = random.randint(0, len(dataset) - 1)
        random_poem = dataset[random_index]['text']  
        
        input_ids = tokenizer.encode(random_poem, return_tensors='pt', truncation=True, max_length=20)
        
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        
        output = model.generate(input_ids, attention_mask=attention_mask, max_length=60, temperature=0.7)

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_texts.append(generated_text)



c:\Users\sin2x\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pa

In [12]:
print(generated_texts)

['На камень жизни роковой\nПриродою заброшен,\nМладенец пылкий и живой\n0|2|И , как в детстве , в детстве , в детстве , В детстве , в детстве , в детстве , в детстве , В детстве , в детстве , в детстве , в детстве ,', 'Как летней иногда порою\nВдруг птичка в комнату влетит\nИ жизнь и свет внесет в нее .|1|2|И в этом мире , где все на свете , И в этом мире , где все на свете , И в этом мире , где все на свете , И в', 'Уже великое небесное светило,\nЛиюще с высоты обилие и свет,\nНачертавший путь к нам , -|0|3|Ибо , как сказано в Евангелии , « не введи нас в искушение , но избавь нас от лукавого ». И', '\nМолчи, скрывайся и таи\nИ чувства и мечты свои –\nПускай в них не осталось следа .|0|1|И пусть не будет больше слез .|1|1|И пусть не будет больше боли .|0|2|Молчи , скрывайся и', 'Как часто, бросив взор с утесистой вершины,\nСажусь задумчивый в тени древес густой .|1|2|И , как в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в д

In [ ]:
import re
pattern = r'-\|(.*?)\|(.*?)\|'

def clean_and_replace(lines):
    cleaned_lines = []
    for line in lines:
        line_replaced = re.sub(r'[|\d]+', '\n', line)
        cleaned_lines.append(line_replaced)
    return cleaned_lines

cleaned_texts = clean_and_replace(generated_texts)
for t in cleaned_texts:
    print("Стихотворение ----------------")
    print(t)

Стихотворение ----------------
В те дни кроваво-роковые,
Когда, прервав борьбу свою,
В ножнах , как в тисках ,
В те дни кроваво - роковые , когда , прервав борьбу свою ,
В ножнах , как в ти
Стихотворение ----------------
Давно ль, давно ль, о Юг блаженный,
Я зрел тебя лицом к лицу ?
Ты , как я , был , как я , и как ты , был .
Ты , как я , был , как я , и как ты
Стихотворение ----------------
Не в первый раз волнуется Восток,
Не в первый раз Христа там распинают,

Не в первый раз в России гибнут люди .
Не в первый раз в России гибнут люди .
Не в первый раз в России
Стихотворение ----------------
Когда осьмнадцать лет твои
И для тебя уж будут сновиденьем, –
С любовью, с нежностью , с тоской , с тоской , –
И , как в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в
Стихотворение ----------------
Спешу поздравить с неудачей:
Она – блистательный успех,
Для вас почетна наикрасивейшая .
И , как я уже сказал , я не могу не заметить , что вы , как и я , не може

In [25]:
for gen_poem in generated_texts:
    print("Стихотворение ----------------")
    print(gen_poem)


Стихотворение ----------------
В те дни кроваво-роковые,
Когда, прервав борьбу свою,
В ножнах , как в тисках ,|0|3|В те дни кроваво - роковые , когда , прервав борьбу свою ,|1|3|В ножнах , как в ти
Стихотворение ----------------
Давно ль, давно ль, о Юг блаженный,
Я зрел тебя лицом к лицу ?|1|2|Ты , как я , был , как я , и как ты , был .|0|2|Ты , как я , был , как я , и как ты
Стихотворение ----------------
Не в первый раз волнуется Восток,
Не в первый раз Христа там распинают,
0|1|Не в первый раз в России гибнут люди .|1|1|Не в первый раз в России гибнут люди .|0|1|Не в первый раз в России
Стихотворение ----------------
Когда осьмнадцать лет твои
И для тебя уж будут сновиденьем, –
С любовью, с нежностью , с тоской , с тоской , –|0|3|И , как в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в детстве , в
Стихотворение ----------------
Спешу поздравить с неудачей:
Она – блистательный успех,
Для вас почетна наикрасивейшая .|1|2|И , как я уже сказал , я не могу не за

In [ ]:
import sacrebleu

def calculate_chrf(reference_texts, generated_texts):
    chrf_score = sacrebleu.corpus_chrf(generated_texts, reference_texts, char_order=6)
    return chrf_score.score

def calculate_perplexity(model, tokenizer, texts):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors="pt")
            outputs = model(**inputs, labels=inputs["input_ids"])
            total_loss += outputs.loss.item()
    
    avg_loss = total_loss / len(texts)
    perplexity = torch.exp(torch.tensor(avg_loss))
    return perplexity.item()

def calculate_novelty(reference_texts, generated_texts):
    reference_set = set(reference_texts)
    unique_generated = set(generated_texts)
    
    novelty_score = len(unique_generated - reference_set) / len(unique_generated) if unique_generated else 0
    return novelty_score

def calculate_distinct_n(generated_texts, n=2):
    ngrams = set()
    for text in generated_texts:
        tokens = text.split()  
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.add(ngram)
    
    distinct_n_score = len(ngrams) / sum(len(text.split()) for text in generated_texts)
    return distinct_n_score

def llm_as_judge(model, tokenizer, texts):
    scores = []
    pattern = re.compile(r'\d+') 

    for text in texts:
        prompt = f"Оцените качество следующего стихотворения от 1 до 10:\n{text}"
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**inputs)
        score_str = tokenizer.decode(outputs[0], skip_special_tokens=True)

        match = pattern.search(score_str)
        if match:
            score = int(match.group())
            score = max(1, min(10, score))
            scores.append(score)
        else:
            print("Не удалось найти оценку в ответе.")
            scores.append(5)

    average_score = sum(scores) / len(scores) if scores else 0
    return average_score


def evaluate_model(model, tokenizer, reference_texts, generated_texts):
    chrf_score = calculate_chrf(reference_texts, generated_texts)
    perplexity_score = calculate_perplexity(model, tokenizer, generated_texts)
    distinct_n_score = calculate_distinct_n(generated_texts)
    novelty_score = calculate_novelty(reference_texts, generated_texts)
    llm_judge_score = llm_as_judge(model, tokenizer, generated_texts)

    print(f"chrF++: {chrf_score:.4f}")
    print(f"Perplexity: {perplexity_score:.4f}")
    print(f"Distinct-n: {distinct_n_score:.4f}")
    print(f"Novelty: {novelty_score:.4f}")
    print(f"LLM-as-judge score: {llm_judge_score:.4f}")

In [ ]:
reference_10_texts = [row['text'] for row in dataset.select(range(10))]

In [ ]:
evaluate_model(model, tokenizer, reference_10_texts, cleaned_texts)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chrF++: 3.8226
Perplexity: 165.1382
Distinct-n: 0.5678
Novelty: 1.0000
LLM-as-judge score: 1.0000


In [ ]:
def generate_poem(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
model_name = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
modelWithoutDPO = AutoModelForCausalLM.from_pretrained(model_name).to(device)

c:\Users\sin2x\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sin2x\.cache\huggingface\hub\models--ai-forever--rugpt3small_based_on_gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [39]:
prompt = "Напиши стихотворение про лес"
print(generate_poem(modelWithoutDPO, tokenizer, prompt))

Напиши стихотворение про лес,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в лесу, и в лесу,
		И в
